In [ ]:
# ####################################################################################
# THIS IS A OPEN AI GPT 3.5 turbo based approach.
# IT IS EXTREMELY RATE LIMIT DEPENDENT AND UP TO THE WHIM'S OF OPENAI'S LIMITS
#
# IT MAY OR MAY NOT WORK ON THE FIRST GO, PROVIDED YOU HAVE A WORKING API KEY.
# ####################################################################################

In [15]:
# Imports and Setup sections
import os
from ultralytics import YOLO
from PIL import Image
import torch
import openai
from IPython.display import display, Markdown
import ipywidgets as widgets

# Set your OpenAI key
openai.api_key = os.getenv("OPENAI_API_KEY")  # Or paste it like: "sk-..."

# Load YOLOv8 model (offline from models/ directory)
yolo_models = {
    "nano": YOLO("models/yolov8n.pt"),
    "small": YOLO("models/yolov8s.pt"),
    "medium": YOLO("models/yolov8m.pt"),
    "large": YOLO("models/yolov8l.pt"),
    "xlarge": YOLO("models/yolov8x.pt")
}
# Select default model variant
yolo_model = yolo_models["xlarge"]

In [3]:
# 2. Ingredient Detection from Image Folder using YOLO (Gimmicky)

def detect_ingredients_from_images(folder_path="Available_Ingredients"):
    detected = set()
    for file in os.listdir(folder_path):
        if file.lower().endswith((".jpg", ".jpeg", ".png")):
            results = yolo_model(os.path.join(folder_path, file))
            for r in results:
                names = r.names
                for c in r.boxes.cls:
                    detected.add(names[int(c)])
    return sorted(detected)

In [16]:
# 3. GPT API-Based Recipe Generation via prompt and ingredients.

client = openai.OpenAI()

def generate_recipe_gpt(ingredients, dish_type="main course"):
    prompt = (
        f"You are a professional chef AI. Create a detailed {dish_type} recipe using the following ingredients:\n"
        f"{ingredients}.\n"
        f"Include a title, a list of ingredients, and numbered cooking steps.\n"
    )

    try:
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "You generate well-structured recipes."},
                {"role": "user", "content": prompt}
            ],
            temperature=0.7
        )
        return response.choices[0].message.content.strip()

    except Exception as e:
        return f"❌ Error generating recipe: {e}"

In [18]:
# 4. Combine Image Detected + Manually input Ingredients

manual_input = widgets.Textarea(
    placeholder="Add undetected items here (e.g. salt, pepper, spices)...",
    description="Extra Items:",
    layout=widgets.Layout(width="100%", height="60px")
)

dish_dropdown = widgets.Dropdown(
    options=["main course", "dessert", "salad", "vegan", "quick meal"],
    value="main course",
    description="Dish Type:"
)

generate_btn = widgets.Button(description="Generate Recipe", button_style="success")
output_area = widgets.Output()

In [20]:
# 5. Button Callback to Run Detection + Generation using widgets library

def on_generate(b):
    with output_area:
        output_area.clear_output()
        display(Markdown("### 🧑‍🍳 Detecting ingredients from images..."))
        detected = detect_ingredients_from_images()
        combined_ingredients = detected + manual_input.value.split(',')
        cleaned = [i.strip() for i in combined_ingredients if i.strip()]
        final_ingredient_list = ", ".join(cleaned)
        
        display(Markdown(f"**Ingredients Used:** {final_ingredient_list}"))
        display(Markdown("### 📜 Generating recipe using GPT..."))
        recipe = generate_recipe_gpt(final_ingredient_list, dish_dropdown.value)
        display(Markdown(recipe))

generate_btn.on_click(on_generate)

In [22]:
# 6. Main Running code to bring functionalities to life.

ui = widgets.VBox([
    widgets.HTML("<h2>🧠 AI Cooking Assistant (Prototype)</h2>"),
    widgets.HTML("<p><b>Detected from:</b> <code>Available_Ingredients/</code></p>"),
    manual_input,
    dish_dropdown,
    generate_btn,
    output_area
])

display(ui)
